# Spark DataFrame Missing Data

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("spark://spark:7077").appName("Operations").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/30 22:35:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.csv("/data/ContainsNull.csv",header=True, inferSchema=True)

In [4]:
df.show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



## Dropping Null Values

In [11]:
# Use .na.drop() to drop rows with any null values
df.na.drop().show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp4|Cindy|456.0|
+----+-----+-----+



In [12]:
# Use .na.drop(thresh=<n>) to drop rows with at least <n> null values
df.na.drop(thresh=2).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [13]:
# Use .na.drop(how=<"any"|"all">) to drop rows with any null values or all null values
df.na.drop(how="all").show()


+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John| null|
|emp2| null| null|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [14]:
# Use .na.drop(subset=<[cols]>) to drop rows with null values in subset of columns
df.na.drop(subset="Sales").show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [ ]:
# Filling Null Values

In [10]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Sales: double (nullable = true)



In [15]:
# Spark knows to automatically fill string columns with string values amd numeric columns with numeric values
df.na.fill(0).show()
# Does not fill string column

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|  0.0|
|emp2| null|  0.0|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+



In [17]:
# Fill specific columns with specific value
df.na.fill("No Name", subset=["Name"]).show()

+----+-------+-----+
|  Id|   Name|Sales|
+----+-------+-----+
|emp1|   John| null|
|emp2|No Name| null|
|emp3|No Name|345.0|
|emp4|  Cindy|456.0|
+----+-------+-----+



In [18]:
# Using computed mean value to fill column
from pyspark.sql.functions import mean

In [19]:
mean_val = df.select(mean(df["Sales"])).collect()

In [20]:
# .collect() returns list of rows
mean_val

[Row(avg(Sales)=400.5)]

In [23]:
print(mean_val[0][0])
mean_sales = mean_val[0][0]

400.5


In [25]:
df.na.fill(mean_sales, subset=["Sales"]).show()

+----+-----+-----+
|  Id| Name|Sales|
+----+-----+-----+
|emp1| John|400.5|
|emp2| null|400.5|
|emp3| null|345.0|
|emp4|Cindy|456.0|
+----+-----+-----+

